---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    import pandas as pd

    file = 'university_towns.txt'
    f = open(file)
    
    States = list()
    Regions = list()
 
    for line in f:
        if 'edit' in line:
            ind = line.index('[')
            state = line[:ind]
        else:
            try:
                ind = line.index('(')
                region = line[:ind].strip()
            except:
                region = line.strip()
            States.append(state)
            Regions.append(region)
    dfUT = pd.DataFrame()
    dfUT['State'] = States
    dfUT['RegionName'] = Regions
    # dfUT.drop_duplicates()
    return dfUT

In [3]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    dfA = pd.read_excel('gdplev.xls', skiprows=5)
    dfB = dfA[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']]  #, 'GDP in billions of chained 2009 dollars.1'
    dfB.columns=['quarter', 'GDP chained 2009']
    dfB['GDP chained 2009 prevQuarter'] = dfB['GDP chained 2009'].shift()
    for i, name in enumerate(dfB['GDP chained 2009 prevQuarter']):    
        if dfB.loc[i, 'GDP chained 2009 prevQuarter'] < dfB.loc[i, 'GDP chained 2009']:
            dfB.loc[i, 'Qchange'] = 1
        else: dfB.loc[i, 'Qchange'] = 0
    dfB['Qchange']
    dfC = dfB.copy().iloc[2:]
    dfD = dfC.copy().loc[dfC.index >= 214,:]
    dfD['followingQchange'] = dfD['Qchange'].shift(periods=-1)
    return dfD[(dfD['Qchange']==0) & (dfD['followingQchange']==0)].iloc[0,:]['quarter']

In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    dfA = pd.read_excel('gdplev.xls', skiprows=5)
    dfB = dfA[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']]  #, 'GDP in billions of chained 2009 dollars.1'
    dfB.columns=['quarter', 'GDP chained 2009']
    dfB['GDP chained 2009 prevQuarter'] = dfB['GDP chained 2009'].shift()
    for i, name in enumerate(dfB['GDP chained 2009 prevQuarter']):    
        if dfB.loc[i, 'GDP chained 2009 prevQuarter'] < dfB.loc[i, 'GDP chained 2009']:
            dfB.loc[i, 'Qchange'] = 1
        else: dfB.loc[i, 'Qchange'] = 0
    dfB['Qchange']
    dfC = dfB.copy().iloc[2:]
    dfC.head()
    dfD = dfC.copy().loc[dfC.index >= dfC[dfC['quarter']=='2008q3'].index.values[0], :]
    dfD['followingQchange'] = dfD['Qchange'].shift(periods=-1)
    return (dfD[(dfD['Qchange']==1) & (dfD['followingQchange']==1)].iloc[1,:]['quarter'])

In [5]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    dfA = pd.read_excel('gdplev.xls', skiprows=5)
    dfB = dfA[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']]  #, 'GDP in billions of chained 2009 dollars.1'
    dfB.columns=['quarter', 'GDP chained 2009']
    dfB['GDP chained 2009 prevQuarter'] = dfB['GDP chained 2009'].shift()
    for i, name in enumerate(dfB['GDP chained 2009 prevQuarter']):    
        if dfB.loc[i, 'GDP chained 2009 prevQuarter'] < dfB.loc[i, 'GDP chained 2009']:
            dfB.loc[i, 'Qchange'] = 1
        else: dfB.loc[i, 'Qchange'] = 0
    dfB['Qchange']
    dfC = dfB.copy().iloc[2:]
    dfC.head()
    dfD = dfC.copy().loc[dfC.index >= dfC[dfC['quarter']=='2008q3'].index.values[0], :]
    dfE = dfD.set_index('quarter')
    minGDP = dfE.loc[get_recession_start():get_recession_end(), 'GDP chained 2009'].min()
    return dfE[dfE['GDP chained 2009']==minGDP].index.astype(str)[0]

In [6]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np

    dfA = pd.read_csv('City_Zhvi_AllHomes.csv')
    dfA.replace({'State':states}, inplace=True)
    dfA.set_index(['State', 'RegionName'], inplace=True)
    # make a dataframe in place, drop years from 1996-2000
    columnsToDrop = dfA.loc[:, '1996-04':'1999-12'].columns.tolist()

    # dataframe with remaining intro columns
    dfA.drop(columnsToDrop, axis=1, inplace=True)

    # convert 3 month periods to quarters by splicing the year (string.split)
    years = list(range(2000, 2017))
    months = list(range(1,13))

    dfB = dfA.copy()
    dfC = pd.DataFrame()
    for year in years:
        tempQ1 = []
        tempQ2 = []
        tempQ3 = []
        tempQ4 = []
        for month in months:   
            if month <= 3:
                tempQ1.append(str(year)+'-'+('{:02}'.format(month)))
            elif (month >= 4 and month <= 6):
                tempQ2.append(str(year)+'-'+('{:02}'.format(month)))
            elif (month >= 7 and month <= 9):
                tempQ3.append(str(year)+'-'+('{:02}'.format(month)))
            elif month >= 10:
                tempQ4.append(str(year)+'-'+('{:02}'.format(month)))        
        yearQ1name = str(year)+'q1'
        dfC[yearQ1name] = dfB[tempQ1].mean(axis=1)
        # except: continue
        yearQ2name = str(year)+'q2'
        dfC[yearQ2name] = dfB[tempQ2].mean(axis=1)
        # except: continue
        yearQ3name = str(year)+'q3'
        try: dfC[yearQ3name] = dfB[tempQ3].mean(axis=1)
        except: dfC[yearQ3name] = dfB[tempQ3[:2]].mean(axis=1)
        yearQ4name = str(year)+'q4'
        try: dfC[yearQ4name] = dfB[tempQ4].mean(axis=1)
        except: continue
    return dfC

In [7]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    import numpy as np
    import scipy.stats as stats

    dfUT = get_list_of_university_towns()
    QrecStart = get_recession_start()
    QrecEnd = get_recession_end()
    QrecBtom = get_recession_bottom()
    dfHD = convert_housing_data_to_quarters()

    colList = dfHD.columns.tolist()
    indexes = [n for n, x in enumerate(colList) if QrecStart in x]
    Q_bfr_rec_start = colList[indexes[0]-1]
    dfHD['PriceRatio'] = dfHD[Q_bfr_rec_start]/dfHD[QrecBtom]

    dfUT.set_index(['State', 'RegionName'], inplace=True)
    dfUT['UT']=1
    dfMerge = pd.merge(dfHD, dfUT, how='left', left_index=True, right_index=True)
    dfMerge2 = dfMerge.copy()
    dfMerge2['UT'].fillna(value=0, inplace=True)
    dfMerge3 = dfMerge2.loc[:, ['PriceRatio','UT']]

    nonUniv = dfMerge3[dfMerge3['UT']==0]
    Univ = dfMerge3[dfMerge3['UT']==1]

    tResult = stats.ttest_ind(nonUniv['PriceRatio'], Univ['PriceRatio'], nan_policy='omit')
    p = tResult.pvalue
    different = p < 0.01
    if nonUniv['PriceRatio'].mean() < Univ['PriceRatio'].mean():
        better = 'non-university town'
    else: better = 'university town'
        
    tupleAnswer = (different, p, better)
    return tupleAnswer